In [1]:
import numpy as np
import tensorflow as tf

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# from plotly.offline import init_notebook_mode
# from IPython.display import display, HTML

# init_notebook_mode()
# display(HTML('<script src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/3.2.2/es5/tex-mml-svg.min.js" integrity="sha512-4MXl9OmsJPCU3LySQiKq4baSCSNFha8CBJL7NVSmN+WJJNLhJUycJgGdxJlxAY2ih8jhP9juGYq2ThUihYBKIw==" crossorigin="anonymous" referrerpolicy="no-referrer"></script>'))

In [2]:
num_quasimomenta = 99
max_momentum_site = 9

max_momentum_site_change = 2 * max_momentum_site
num_momentum_sites = (max_momentum_site_change + 1) ** 2

In [3]:
quasimomentum = np.mgrid[0:2:num_quasimomenta*1j, 1:1:1j].reshape(2,-1).T # [k_L]

momentum_site = np.mgrid[-max_momentum_site:max_momentum_site+1:1,
                         -max_momentum_site:max_momentum_site+1:1].reshape(2,-1).T # Fundamental: k_L

momentum = 2*momentum_site + quasimomentum[:, np.newaxis]
kinetic = np.linalg.norm(momentum, axis=-1)**2

momentum_site_change = momentum_site[:, np.newaxis] - momentum_site

In [4]:
def render_momentum_site_potential(attractive_strength, repulsive_strength, repulsive_spread):
    attractive_potential = -1/16 * attractive_strength * np.pad(np.array([
        [1, 0, 2, 0, 1],
        [0, 0, 0, 0, 0],
        [2, 0, 4, 0, 2],
        [0, 0, 0, 0, 0],
        [1, 0, 2, 0, 1]
    ]), max_momentum_site_change - 2)

    momentum_site = np.meshgrid(*(np.arange(-max_momentum_site_change, max_momentum_site_change+1),)*2)
    repulsive_potential = repulsive_strength * (np.sqrt(2*np.pi) * repulsive_spread)**2 * np.exp(-np.pi * (np.sqrt(2*np.pi) * repulsive_spread)**2 * (momentum_site[0]**2 + momentum_site[1]**2))

    return attractive_potential + repulsive_potential

In [5]:
def compute_spatial_potential(momentum_site_potential, num_points=0):
    padded_momentum_site_potential = np.pad(momentum_site_potential, num_points)

    position = np.fft.fftshift(np.fft.fftfreq(n=padded_momentum_site_potential.shape[0])) # [1/k_L]
    spatial_potential = np.real(np.fft.fftshift(np.fft.fft2(np.fft.ifftshift(padded_momentum_site_potential))))

    return position, spatial_potential

In [20]:
def compute_dispersion(momentum_site_potential):
    potential = momentum_site_potential[
        momentum_site_change[:, :, 0] + max_momentum_site_change,
        momentum_site_change[:, :, 1] + max_momentum_site_change,
    ]

    energy = tf.linalg.diag(kinetic) + potential[np.newaxis, ...]

    energy_dispersion, state_dispersion = tf.linalg.eigh(energy)
    
    return energy_dispersion, state_dispersion

In [7]:
def compute_tight_binding_parameters(dispersion):
    width = np.max(dispersion[:, 2]) - np.min(dispersion[:, 2])
    zero = np.max(dispersion[:, 1])
    tunneling = width/2

    return zero, tunneling

In [8]:
def compute_tight_binding_dispersion(zero, tunneling):
    return np.array([-1, 0, +1])[:, np.newaxis] * 2*tunneling * np.abs(np.cos(quasimomentum[:, 0] * np.pi/2)) + zero

In [23]:
attractive_strength = 50
repulsive_strength = 2
repulsive_spread = 0.128

momentum_site_potential = render_momentum_site_potential(attractive_strength, repulsive_strength, repulsive_spread)

position, spatial_potential = compute_spatial_potential(momentum_site_potential, num_points=100)
energy_dispersion, state_dispersion = compute_dispersion(momentum_site_potential)
zero, tunneling = compute_tight_binding_parameters(dispersion)
tight_binding_dispersion = compute_tight_binding_dispersion(zero, tunneling)

In [24]:
figure = make_subplots(
    rows=1, cols=2,
    subplot_titles=(
        rf"$\text{{Potential }} V(x)$",
        rf"$\text{{Dispersion }} E(k_x) \longrightarrow (E_0/E_R={zero:.3f}, t/E_R={tunneling:.3f})$"
    )
)

for band in range(3):
    figure.add_trace(go.Scatter(x=quasimomentum[:, 0], y=energy_dispersion[:, band], name=fr"$\text{{OL}}: {['-', '0', '+'][band]}$"), row=1, col=2)

for band in range(3):
    figure.add_trace(go.Scatter(x=quasimomentum[:, 0], y=tight_binding_dispersion[band], name=fr"$\text{{TB}}: {['-', '0', '+'][band]}$", line=dict(dash='dash')), row=1, col=2)

figure.update_layout(
    xaxis2=dict(title=r"$\text{Quasimomentum }k_x/k_L$"),
    yaxis2=dict(title=r"$\text{Energy }E/E_R$"),
)

figure.add_trace(go.Scatter(x=position, y=spatial_potential[:, spatial_potential.shape[1]//2], name=""), row=1, col=1)
figure.update_layout(
    xaxis=dict(title=r"$\text{Position }xk_L$"),
    yaxis=dict(title=r"$\text{Energy }E/E_R$"),
)

figure.update_layout(
    title_text=fr"$\text{{Analysis @ }}(V_-/E_R={attractive_strength}; V_+/E_R={repulsive_strength}, \sigma_+k_L={repulsive_spread})$",
    autosize=False, width=1500, height=750)

figure.show()

In [35]:
state_dispersion[0]

<tf.Tensor: shape=(361, 361), dtype=float64, numpy=
array([[-1.09655554e-09,  4.27345307e-08, -6.66413939e-08, ...,
        -5.88586491e-14, -3.79322535e-14,  3.28633600e-14],
       [ 3.05757783e-09,  1.37969641e-07,  2.15641329e-07, ...,
        -7.99871298e-14, -2.53185875e-15,  2.05735214e-15],
       [-9.96860535e-09,  3.99487567e-07, -6.23683681e-07, ...,
         3.44477763e-14,  3.12202735e-13, -3.29346827e-13],
       ...,
       [-3.08417197e-09,  1.38779549e-07, -2.16894499e-07, ...,
        -4.29918844e-04, -3.18801354e-02, -3.00541537e-02],
       [ 1.07893759e-09,  4.27358314e-08,  6.66610114e-08, ...,
        -1.72603473e-02,  1.40251897e-03,  1.32218763e-03],
       [-2.45685308e-10,  1.10275178e-08, -1.72110941e-08, ...,
         1.37561582e-03,  7.26108839e-01,  6.84519885e-01]])>

In [55]:
figure = go.Figure(data=[go.Surface(z=np.abs(np.fft.fftshift(np.fft.fft2(np.fft.ifftshift(
    np.reshape(state_dispersion[0, :, 0], (max_momentum_site_change + 1, max_momentum_site_change + 1))
))))) ])

figure.update_layout(autosize=False, width=750, height=750)
figure.show()